The following code is written to perform Object Detection on Google Colab.


Import Tensorflow and check its version

In [ ]:
!pip install tf-nightly-gpu-2.0-preview

import tensorflow as tf
print(tf.__version__)
import tensorflow_hub as hub

Import essential Python Libraries

In [ ]:
# For downloading the image
import matplotlib.pyplot as plt
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

# FOr drwaing on the image
import numpy as np
from PIL import Image
from PIL import ImageDraw
from PIL import ImageColor
from PIL import ImageFont
from PIL import ImageOps

#TIME
import time
#Check Available GPU
print("Number of GPU Available", tf.test.gpu_device_name())

Define Functions for display image, download and resize image & draw boounding box on image.

In [ ]:
def display_image(image):
  flt = plt.figure(figsize=(20,15))
  plt.grid(False)
  plt.imshow(image)

def download_and_resize_image(url, new_height = 256, new_width = 256, display = False):
  _, filename = tempfile.mkstemp(suffix = ".jpg")
  response = urlopen(url)
  image_data = response.read()
  image_data = BytesIO(image_data)
  pil_image = Image.open(image_data)
  pil_image = ImageOps.fit(pil_image, (new_height, new_width), Image.ANTIALIAS)
  pil_image_rgb = pil_image.convert("RGB")
  pil_image_rgb.save(filename, format = "JPEG", quality = 90)
  print("Image downloaded to %s"%filename)
  if display:
    display_image(pil_image)
  return filename

def draw_bounding_box_on_image(image, ymin, xmin, ymax, xmax, color, font, thickness = 4, display_str_list= ()):
  draw = ImageDraw.Draw(image)
  im_width, im_height = image.size
  (left, right, top, bottom) = (im_width*xmin, im_width*xmax, im_height*ymin, im_height*ymax)
  draw.line([(left,top),(left,bottom),(right,bottom),(right,top),(left,top)], width = thickness, fill = color)

  display_str_height = [font.getsize(ds)[1] for ds in display_str_list]
  total_display_str_height = (1+2*0.05)*sum(display_str_height)
  
  if top > total_display_str_height:
    text_bottom = top 
  else:
    text_bottom = bottom + total_display_str_height

  for display_str in display_str_list[::-1]:
    text_width, text_height = font.getsize(display_str)
    margin = np.ceil(0.05*text_height)
    draw.rectangle([(left, text_bottom-text_height-2*margin),(left+text_width+2*margin,text_bottom)],fill = color)
    draw.text([left+margin, text_bottom-text_height+margin],display_str,fill = "black",font = font)
    text_bottom -= text_height - 2*margin

def draw_boxes(image, boxes, class_names, scores, max_boxes=10, min_score = 0.1):
    colors = list(ImageColor.colormap.values())
    font = ImageFont.load_default()

    for i in range(min(boxes.shape[0], max_boxes)):
      if scores[i]>=min_score:
        ymin, xmin, ymax, xmax = tuple(boxes[i])
        display_str = "{} : {}%".format(class_names[i].decode("ascii"), int(100* scores[i]))

        color = colors[hash(class_names[i]) % len(colors)]
        
        
        image_pil = Image.fromarray(image).convert("RGB")
        
        draw_bounding_box_on_image(
            image_pil,
            ymin,
            xmin,
            ymax,
            xmax,
            color,
            font,
            display_str_list = [display_str]
        )
        np.copyto(image, np.array(image_pil))
    return image

Download image from internet for Testing.

In [ ]:
image_url = "https://farm1.staticflickr.com/4032/4653948754_c0d768086b_o.jpg" 
download_image_path = download_and_resize_image(image_url, 1200, 856, True)

Download Model to perform object detection


In [ ]:
module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1" #@param ["https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1", "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"]

detector = hub.load(module_handle).signatures['default']

Define Function to load image.

In [ ]:
def load_img(path):
  img = tf.io.read_file(path)
  img = tf.image.decode_jpeg(img, channels=3)
  return img

Define function to perform Object Detection

In [ ]:
def run_detector(detector, path):
  img = load_img(path)
  converted_img = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
  start_time = time.time()
  results = detector(converted_img)
  end_time = time.time()
  
  result = {key: value.numpy() for key, value in results.items()}
  
  print("Found %d objects"%len(result["detection_scores"]))
  print("Inference Time : ", end_time-start_time)

  image_with_boxes = draw_boxes(img.numpy(), result["detection_boxes"], result["detection_class_entities"], result["detection_scores"])
  display_image(image_with_boxes)

Run object detection on testing image.


In [ ]:
run_detector(detector, download_image_path)

Run object detection on three images from internet

In [ ]:
image_urls = ["https://farm7.staticflickr.com/8092/8592917784_4759d3088b_o.jpg",
              "https://farm6.staticflickr.com/2598/4138342721_06f6e177f3_o.jpg",
              "https://c4.staticflickr.com/9/8322/8053836633_6dc507f090_o.jpg"]
for image_url in image_urls:
  image_path = download_and_resize_image(image_url, 640, 480)
  run_detector(detector, image_path)


Thanks. 
VIPUL KAMBOJ